In [26]:
import cv2 as cv 
import numpy as np 
import os

#Orientation Checker Remains to be added, Use Images of correct orientation for now

def reshapeImg(img, scale = 0.1): 
    width = int(img.shape[1]*scale)
    height = int(img.shape[0]*(scale))
    dimensions = (width,height)
    return cv.resize(img,dimensions,interpolation=cv.INTER_AREA)

def compressImg(directory):
    filenames = os.listdir(directory)
    for image in filenames:
        img = cv.imread(f'{directory}/{image}')
        
        img_reshape = reshapeImg(img)
        img_gray = cv.cvtColor(img_reshape,cv.COLOR_BGR2GRAY)
        
        kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE,(1,1))
        
        sharpened_image = cv.filter2D(img_gray, -1, kernel)
        if not os.path.exists("compressed"):
              os.makedirs("compressed")
        
        cv.imwrite(f"compressed/{image}",sharpened_image)
    print("Done")

compressImg("C:/Users/tusha/Desktop/NCVP/Train_Samples_V0")

Done


In [28]:
import cv2
import numpy as np
import PIL.Image as Image
import os




def save_tf_answers(directory, qno = -1):
    filenames = os.listdir(directory)
    splitted_images = 0
    total_images = 0
    for image_name in filenames:
        try:
            img = cv2.imread(f'{directory}/{image_name}') ##
            total_images += 1
            img = (img[:, 3*img.shape[1]//5:])
            H,W = img.shape[0], img.shape[1]
        
            print(img.shape)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            #Image.fromarray(gray).show()
            canny = cv2.Canny(gray, 50, 200, None, 3)
            #Image.fromarray(dst).show()
            
            try:
                lines = cv2.HoughLines(canny, 1, np.pi/180, 200)
                lines = np.reshape(lines, (lines.shape[0], 2))
            
                
                vertical_lines = []
                
                for  i in lines:
                    if abs(np.cos(i[1]))>0.99:
                        if(np.cos(i[1])<0):
                            i[0] = abs(i[0])
                            i[1] = np.pi + i[1]
                            vertical_lines.append(i)
                        else:
                            vertical_lines.append(i)
                            
                vertical_lines = np.array(vertical_lines)
                vertical_lines = vertical_lines[vertical_lines[:, 0].argsort()]
                v_edges = [vertical_lines[0],]
                
                for i in range(1,len(vertical_lines)):
                    if abs(vertical_lines[i][0] - v_edges[len(v_edges)-1][0]) < W//3: #Magic number
                        pass
                    else:
                        v_edges.append(vertical_lines[i])
            
                v_edges = np.array(v_edges, dtype = int)
        
            
                TFcol = gray[:, v_edges[0][0]:v_edges[1][0]]  #Image of the True/False Column
                
            except:
                print(f"ERROR, Image: {image_name} is Not Clear")
                continue
                
            cannyTF = cv2.Canny(TFcol, 100, 150, None, 3) 
        
            boxTF = cannyTF.copy()  # To show locations of True/False boxes after processing canny image of True/False Column Image
            horizontal = []         # Tracks the horizontal lines
            kernel = 5              # Determines sensitivity in Line detection, ( 3 - 7) is the ideal range
            
            for i in range(kernel,cannyTF.shape[0]-kernel):
                h_line = False
                for j in range(1, cannyTF.shape[1]-1):
                    for k in range(-1*kernel, kernel):
                        if (cannyTF[i+k][j-1] == 255 or cannyTF[i+k][j] == 255 or cannyTF[i+k][j+1] == 255):
                            h_line = True
                            break
                    else:
                        h_line = False
                        break
                    
                    h_line = True
                if (h_line):
                    horizontal.append(i)
                    boxTF[i,:] = 0
                else:
                    boxTF[i, :] = 255
        
        
            
        
            # Image.fromarray(cannyTF).show()
            # Image.fromarray(boxTF).show()
            
            min_lines = []                       # Stores the row of 1 line from each horizontal lines cluster, 
            s, n = 0, 0
            for i in range(len(horizontal)-1):
                if horizontal[i+1]-horizontal[i] <= kernel:
                    s += horizontal[i]
                    n += 1
                    if( i == len(horizontal)-2):
                        min_lines.append(int(s/n))
        
                else:
                    s += horizontal[i]
                    n += 1
                    min_lines.append(int(s/n))
                    if( i == len(horizontal)-2):
                        min_lines.append(horizontal[-1])
                    s = 0
                    n = 0
                    
            #print(horizontal)
        
            #Removes if a line is detected at the top of the Image
            
            if min_lines[0] < 10:
                min_lines = min_lines[1:]
            if (qno < 0):
                pass
            else:
                try:
                    min_lines = min_lines[:qno+1]
                except:
                    pass
            print(f"Image {image_name} splits:", len(min_lines)-1)
        
            
            for i in range(len(min_lines)-1):
                sliced_img = TFcol[min_lines[i]:min_lines[i+1], :]
                folder_path =  f"sliced/{image_name[:-4]}"
                if not os.path.exists(folder_path):
                  os.makedirs(folder_path)
                cv2.imwrite(f"{folder_path}/res_{i+1}.png", sliced_img)
            splitted_images += 1
        except:
            pass

    print(f"Images {splitted_images}/{total_images} splitted")


#Usage save_tf_answers(directory, qno), directory=> folder location containing images, qno=> no. of true/false, set -1 if not known.
#Saves images in sliced/ if not present it will be created automatically
#Use compressed images only from the compressImg() function

save_tf_answers("compressed/", 10)


(783, 276, 3)
Image 20240328_145148.jpg splits: 10
(816, 278, 3)
Image 20240328_145247o.jpg splits: 10
(792, 277, 3)
Image 20240328_145719.jpg splits: 10
(806, 275, 3)
Image 20240328_150204.jpg splits: 10
(792, 276, 3)
Image 20240328_150636.jpg splits: 10
(782, 276, 3)
Image 20240328_150931.jpg splits: 10
(795, 276, 3)
Image 20240328_151020.jpg splits: 10
(805, 277, 3)
Image 20240328_151033.jpg splits: 10
(807, 276, 3)
Image 20240328_151238.jpg splits: 10
(809, 277, 3)
Image 20240328_151304.jpg splits: 10
(802, 275, 3)
Image 20240328_152314.jpg splits: 10
(804, 276, 3)
Image 20240328_152349.jpg splits: 10
(788, 274, 3)
Image 20240328_152408.jpg splits: 10
(788, 276, 3)
Image 20240328_153036.jpg splits: 10
(798, 277, 3)
Image 20240328_153316.jpg splits: 10
(780, 276, 3)
Image 20240328_153338.jpg splits: 10
(786, 274, 3)
Image 20240328_153349.jpg splits: 10
(783, 275, 3)
Image 20240328_153433.jpg splits: 10
(792, 276, 3)
Image 20240328_153456.jpg splits: 10
(778, 275, 3)
Image 20240328_1